## Imports

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
# Crawlers
import requests
from SubCrawlers.BoatosCrawler import Boatos
from bs4 import BeautifulSoup
from datetime import datetime
from time import time as now
# Limpeza de textos
import nltk
import re
import json

## Utils

In [2]:
BASE_DE_DADOS_ATUAL = "../base_completa/boatos_br_corpus.json"
# Ler Base de dados
base_carregada = pd.read_json(BASE_DE_DADOS_ATUAL)

# StopWords
nltk.download('stopwords')
stopwords_portugues = set(nltk.corpus.stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Crawler

### Configurando Crawler

In [3]:
URLS_INICIAIS = [
    "https://www.boatos.org/",
]
ULTIMA_DATA_ATUAL = datetime.fromisoformat(base_carregada["data-publicacao"][0])
modelo = Boatos(urlsIniciais=URLS_INICIAIS)

### Executando busca

In [4]:
data = []
# Buscar noticias novas -> Que não estão na base
data = modelo.scrape(dateToStop=ULTIMA_DATA_ATUAL)

Pagina:  https://www.boatos.org/
Parando Busca...
Tempo gasto:  4.54s


### Normalizando Resultados da busca

#### Logica de limpeza dos texto
* Selecionar apenas letras (remover pontuação, números e outros caracteres)
* Transformar palavras em minusculo
* Remover stopwords
* Manter emojis

In [3]:
def buscaPalavras(texto:str) -> str:
    palavrasMinusculas = re.findall(r'\b[A-zÀ-úü]+\b', texto.lower())
    textoLimpo = [w for w in palavrasMinusculas if w not in stopwords_portugues]
    return " ".join(textoLimpo)

#### Logica de Rotulação das noticias
* Analisar se no texto possui alguma das frases presentes no dicionário criado

In [4]:
# Dicionários
CATEGORIASBOATOSFALSO = [
    "Fake news",
    "fake news",
    "Boato sem comprovação",
    "Golpe", 
    "Enganoso", 
    "Exagerado", 
    "é falso",
    "é falsa",
    "não é verdade",
    "não procede",
    "não há provas"
]
CATEGORIASBOATOSVDD = [
    "Verdadeiro", 
    "Real com erros",
    "É fato",
]

def defineRotulo(categorias):
    for categoria in categorias:
        if categoria in CATEGORIASBOATOSFALSO:
            return 'falso'
        if categoria in CATEGORIASBOATOSVDD:
            return 'verdade'
    return False

def defineRotulobyText(text):
    for categoria in CATEGORIASBOATOSFALSO:
        if categoria in text:
            return 'falso'
    for categoria in CATEGORIASBOATOSVDD:
        if categoria in text:
            return 'verdade'
    return False

def AplicarRotulo(pageData):
    atributos_verificar = ['texto','conclusao','titulo']
    rotulo = defineRotulo(pageData['categorias'])
    if not rotulo:
        for attr in atributos_verificar:
            rotulo = defineRotulobyText(pageData[attr])
            if rotulo != False:
                break
        if not rotulo:
            rotulo = "NDA"
    return rotulo

### Normalização da base

#### Atributos 
- **url**: URL de onde foi obtido o texto.
- **data-publicacao**: Data em que foi publicada a checagem do texto.
- **origem**: 
- **categorias**: Categorias do boato, por exemplo: Política, Saúde, Mundo, entre outras.
- **texto**: Texto original do boato que esta circulando na internet
- **texto-normalizado**: Texto normalizado e limpo.
- **rotulo**: Atributo alvo da previsão, podendo ser Verdadeiro ou Falso.


In [5]:
class BoatosBrData:
    def __init__(self,url,dataPublicacao,origem,categorias,texto,textoNormalizado,rotulo) -> None:
        self.url  = url
        self.datapublicacao = str(dataPublicacao)
        self.origem = origem
        self.categorias = categorias
        self.texto = texto
        self.textoNormalizado = textoNormalizado
        self.rotulo = rotulo
    
    def getData(self) -> dict:
        return {
            'url': self.url,
            'data-publicacao': self.datapublicacao,
            'origem': self.origem,
            'categorias': self.categorias,
            'texto': self.texto,
            'texto-normalizado': self.textoNormalizado,
            'rotulo': self.rotulo,
        }

In [10]:
normalized_data = []
for pageData in data:
    boatosBrData = BoatosBrData(
        url=pageData['url'],
        dataPublicacao=pageData['data-publicacao'],
        origem='boatosorg',
        categorias=pageData['categorias'],
        texto=pageData['referencia'],
        textoNormalizado=buscaPalavras(pageData['referencia']),
        rotulo=AplicarRotulo(pageData)
    )
    normalized_data.append(boatosBrData.getData())

### Salvar Dados obtidos na base

#### Unir dados novos com a base atual

In [11]:
dsNovo = pd.DataFrame(normalized_data)
ds = pd.concat([dsNovo,base_carregada],axis=0).to_dict('records')

#### Criar nova base

In [12]:
with open(f"./base_atualizada/boatos_br_corpus_atualizada.json",'w+',encoding="utf8") as f:
    json.dump(ds,f,indent=3,allow_nan=True,ensure_ascii = False)
    f.close()